# Covid-19 forecasting 

Данный шаблон решения аттестационной задачи предназначен для того, чтобы создать уже готовую структуру, на которую можно нанизывать решение задачи.

Этот шаблон универсален: идейно последовательность решения любой исследовательской задачи будет включать точно такие же шаги. Поэтому в будущем можно ориентироваться на этот шаблон.

При работе очень важно соблюдать некоторые правила оформления работы:    
- Работа пишется в третьем лице, логика построения работы должна примерно соответствовать повествовательной. Представьте, что Вы пишете работу в роли любознательного исследователя, который каждую важную мысль или наблюдение фиксирует у себя в тетради; такой подход позволит сделать чтение исследования приятнее и проще. А это важно - ведь вы пишете и оформляете тетрадь не для собственной гордости, а для того, чтобы эту информацию кто-то мог интерпритировать и использовать.
- Блоки должны всегда быть проверены на последовательность выполнения. Мы уже говорили об особенности Jupyter Notebook и Colab запускать блоки в любом порядке. Когда вы будете возвращаться к своей тетради, важно, чтобы вы могли а) запустить тетрадь и б) увидеть результат. Поэтому После окончания выполнения работы обязательно нужно "прогнать" тетрадь ещё раз - так мы убедимся, что другой человек или мы (в будущем) сможем воспроизвести исследование.
- В конце каждого большого блока/главы (к примеру, EDA) мы должны указывать выводы по проведенной работе. К примеру, логичный вывод "Введения" - фиксация того, что мы сформулировали задачи и цели. Звучит излишне, но в других блоках вывод будет гораздо важнее - если нам нужно будет вспомнить быстро результат работы, мы можем обратиться к выводам по каждой главе и понять, что было сделано и каковы результаты проделанной работы.

При сдаче работы нужно удалить этот блок 😸.

## Введение. Постановка целей и задач.

В блоке введение нам необходимо четко обозначить, **почему эта задача появилась, зачем её важно решить**. 

После того, как мы четко указали на проблему, мы можем сформулировать **цели** исследования и **задачи** исследования. Необходимо всегда помнить, что решение задач ведет к достижению целей, то есть они напрямую взаимосвязаны.

## Импорт библиотек, ознакомление с данными.

### Импорт Датафрейма, библиотек, написание функций.

В этом блоке мы импортируем все, что нам нужно для работы: библиотеки, отдельные функции, сам датасет.

In [1]:
!pip install pmdarima

In [2]:
!pip install fbprophet

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX       # для модели SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose      # для ETS графиков
from pmdarima import auto_arima                              # для поиска ARIMA моделей (подбирает оптимальные параметры для модели)
from fbprophet import Prophet                                # для модели Профет
from statsmodels.tsa.holtwinters import ExponentialSmoothing # для модели Экспоненцальное сглаживание

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import rmse

Importing plotly failed. Interactive plots will not work.


In [4]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df = pd.read_csv(url)

In [5]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


### Знакомство с данными.

Этот блок посвящен первоначальному знакомству с данными

Наша задача - посмотреть на данные методами .info(), а также изучить их визуально.

Будет удобно рассматривать определенный блок - одну страну, один месяц одной страны.

Здесь же надо определиться с описанием данных - лучше всего для себя пояснить, с чем мы имеем дело. В данном случае столбцов слишком много, поэтому необходимо описать только самое важное - максимум 7-8 столбцов.

Как только у нас есть информация, что данные получены без ошибок, а также есть самое общее представление о содержимом, мы можем перейти к следующему пункту..

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149028 entries, 0 to 149027
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    149028 non-null  object 
 1   continent                                   140034 non-null  object 
 2   location                                    149028 non-null  object 
 3   date                                        149028 non-null  object 
 4   total_cases                                 146418 non-null  float64
 5   new_cases                                   146412 non-null  float64
 6   new_cases_smoothed                          145267 non-null  float64
 7   total_deaths                                129414 non-null  float64
 8   new_deaths                                  129610 non-null  float64
 9   new_deaths_smoothed                         145267 non-null  float64
 

In [7]:
df.location.unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethi

## Предобработка данных

### Фильтрация данных

Обычно фильтрацию данных делают позже - когда нужно непосредственно работать с данными. В нашем случае это неудобно: у нас условие, по которому мы смотрим на 1 определенную страну. Поэтому здесь нужно **вывести список стран и выбрать 1 конкретную страну**. 

С другой стороны, столбцов в задании также дали многовато. Поэтому на этом этапе нам нужно избавиться от всех ненужных столбцов. Чтобы понять, какие столбцы нужны - лучше всего вспомнить суть задачи. 


По итогу у нас должен остаться небольшой **датасет** с данными по 1 стране и самыми нужными столбцами.

### Обработка пропусков

В этом блоке мы должны понять - почему у нас есть пропуски. Главный вопрос - какова их природа? 

После того, как мы это поймем, мы можем принять решение относительно заполнения пропусков.

Главный момент: **в рамках этого задания мы не удаляем ничего, связанного с пропусками!**

Главная причина - характер работы с данными: мы работаем в этой задаче с временным промежутком, и нам важно, чтобы не было отсутствующих дат. К примеру, ситуация, когда у нас есть пропуск продолжительностью от 1 до 3 дней в середине временного промежутка, попросту неприемлима.

После обработки пропусков нам обязательно нужно проверить - всё ли в порядке. Выведите данные и проверьте заполненность. Зафиксируйте то, что пропусков больше нет.

### Вывод по этапу.


## EDA или разведочный анализ данных.

В данном блоке наша задача - познакомиться "поближе" с данными, которыми мы располагаем.

Что лучше всего сделать:    
- Сделать столбец с датами индексом;
- Вывести статистику по нужным столбцам;
- Построить графическое отображение столбцов;
- Попробовать определить - есть ли столбцы, которые мы можем отфильтровать (к примеру, самые ранние записи, где информации могло не быть и где были NaN). **- такого может и не быть**


Проделав действия выше, мы выполним минимум, необходимый для дальнейшей работы. 

Помимо этой информации, здесь же нужно всесторонне познакомиться с рассматриваемыми данными в контексте процесса/явления:    
- С чем связан рост или падение метрики в столбце? (почему заболеваемость может расти или падать)
- Как можно охарактеризовать зависимость по метрике? 

Хорошо здесь описать всё, что влияет на метрику, и что может нам помочь определить следующее: можем ли мы достоверно предсказать рост или падение метрики на основании одной переменной? Или здесь нужны методы посложнее, с большим числом входных данных и зависимостей?

Отвечайте на вопросы честно! 

Как только мы поняли, что описанной информации достаточно - можно перейти к следующему пункту.

## Построение моделей, анализ результатов.

Данный блок содержит в себе решение поставленной задачи.

В прошлых блоках мы убедились в целостности данных, их логичности, а также в том, что мы понимаем, с чем мы имеем дело. 

На этом этапе у нас должен быть объект типа DataFrame или Series, где индекс - даты, а в значениях находится нужная информация. То есть временной ряд.

В данном блоке у нас должно быть от трёх до четырех подглав:    
- Разделение датасета на тренировочную и тестовую (валидационную) выборки, проверка на статичность, построение графиков автокорреляции и частичной автокорреляции, разбивка на сезонность и тренд.
- Применение 1 способа прогнозирования;
- Применение 2 способа прогнозирования;
- (Применение 3 способа прогнозирования); **По желанию**

Первый пункт - разделение датасета - универсален, данные из него не нужно обновлять или заного рассчитывать.

Остальные три пункта содержат примерно одинаковый алгоритм действий:    
1. Описываем вкратце модель.
2. Создаем модель.
3. Обучаем модель.
4. Создаем прогноз.
5. Сравниваем прогноз и предсказание. 
6. Рассчитываем качество полученной модели и прогноза.
7. Делаем выводы по прогнозу.


Какие модели у нас есть:
- В рамках курса мы рассмотрели модель ARIMA, её улучшенную версию SARIMA. Обязательно попробуйте использовать эту модель;
- Модель экспаненционного сглаживающего среднего;
- Модель Хольта-Винтерса.

Последние 2 модели мы не изучали, поэтому можно попробовать использовать составные от модели ARIMA, которые мы рассматривали в лекции - AR, MA.

После получения прогноза отобразите графики **прогноз vs факт**, чтобы визуально можно было посмотреть на то, как точно построен прогноз. 

Как понять, подходит ли модель. Попробуйте использовать следующие методы рассчета ошибки:
- MAPE или средняя абсолютная ошибка в процентах.
- RMSE - квадратный корень из среднеквадратичной ошибки.
- MSE - среднеквадратичный корень (связан  с RMSE).

В конце - сравните полученные метрики с ошибками и выберите ту модель, которая точнее всего создает прогноз.

Ещё один важный момент - горизонт прогнозирования и сравнение прогноза с данными. Горизонт планирования нужно сделать небольшим - примерно 10-15 наблюдений (то есть 10-15 дней). И в ошибку нужно сравнивать с таким же числом наблюдений за тот же период по реальным данным!

Также отмечу, что прогноз, скорее всего, будет не самым точным, а возможно, он совсем покажет что-то непонятное. С учетом того, насколько сложен процесс распространения коронавирусной инфекции, можно с уверенностью сказать, что обычный анализ временного ряда врядли даст очень хороший результат. Иначе бы у ученых-прогнозистов не было проблем с предзказанием коронавирусной инфекии 🙌.

## Выводы

В данном блоке опишите общие выводы по работе: что было сделано, что было получено. Помните, отрицательный результат - тоже результат. Поэтому указывайте информацию честно и беспристратно. 